In [1]:
%load_ext autoreload
%autoreload 2

# Comparison of mutational robustness of sRNA's vs. randomly generated RNA

## Imports

In [2]:
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import re
import os
from subprocess import Popen, PIPE
from copy import deepcopy

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from synbio_morpher.srv.io.manage.script_manager import script_preamble
from synbio_morpher.srv.parameter_prediction.simulator import process_raw_stdout
from synbio_morpher.utils.common.setup import expand_model_config, prepare_config, expand_config
from synbio_morpher.utils.evolution.mutation import apply_mutation_to_sequence, get_mutation_type_mapping, reverse_mut_mapping
from synbio_morpher.utils.circuit.agnostic_circuits.circuit_manager import CircuitModeller
from synbio_morpher.utils.misc.type_handling import flatten_listlike, flatten_nested_dict
from synbio_morpher.utils.data.data_format_tools.common import load_multiple_as_list, load_json_as_dict


import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import jax
jax.config.update('jax_platform_name', 'cpu')

jax.devices()

xla_bridge.py:backends():622: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Interpreter INFO
xla_bridge.py:backends():622: Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client' INFO


[CpuDevice(id=0)]

## Load data

In [3]:
fn = '../data/sRNA/merged_inter.csv'
data = pd.read_csv(fn)
try:
    data = data.drop(columns=['Unnamed: 0'])
except:
    pass
data = data[data['Sequence 1'].apply(lambda x: type(x) == str)]
data = data[data['Sequence 2'].apply(lambda x: type(x) == str)]


In [4]:
data

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB,Genome position 1,Genome position 2,Strand 1,Strand 2,Sequence before 1,Sequence before 2,Sequence after 1,Sequence after 2
0,dsrA,dsrB,NCBI:946470,NCBI:946468,sRNA,mRNA,NaN,NaN,NaN,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,TAAATGNAAACAAGTTTTATTTTTATCATCAGAACATTCTAATAAA...,RNAInter,NaN,NaN,NaN,NaN,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,atcgttttcctcctgtggctttgtgccagtgtagaacaatttcgtt...,cgtgaggtcagcgtcgatgtcgatgccctgctggcggcgatcaatg...,NaN
1,dsrA,ilvH,NCBI:946470,NCBI:947267,sRNA,mRNA,NaN,NaN,NaN,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,RNAInter,NaN,NaN,NaN,NaN,ttcatcaccttatccgcaatttttttcgctgacaagaaaatattcg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...,cgtgaggtcagcgtcgatgtcgatgccctgctggcggcgatcaatg...,ATTCGCCACAACAAGGACAGGAAGGTTCGGCGCAAGGAGCCCAAGA...
2,oxyS,fhlA,NCBI:2847701,NCBI:947181,sRNA,mRNA,NaN,NaN,NaN,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,RNAInter,NaN,NaN,NaN,NaN,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,CACCAGATTGCCAAGCTGATGGTCGAGCTGTCCAAATCCCAGGATG...
3,sgrS,hfq,NCBI:4056038,NCBI:948689,sRNA,mRNA,NaN,NaN,NaN,GATGAAGCAAGGGGGTGCCCCATGCGTCAGTTTTATCAGCACTATT...,ATGGCTAAGGGGCAATCTTTACAAGATCCGTTCCTGAACGCACTGC...,RNAInter,NaN,NaN,NaN,NaN,ttgagcagggtgcgcatatgacgacgcgagcagctcaataacgctg...,cgaagcggcagataacctggctgcgtggttgggaaggggttcactg...,ttattctcgccgcgctaaaaagggaacgtatgatctggataatgac...,aaggtttcgggctgtttttttacacggggagccagcgatcctgcgt...
4,oxyS,nusG,NCBI:2847701,NCBI:948485,sRNA,mRNA,NaN,NaN,NaN,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,ATGTCTGAAGCTCCTAAAAAGCGCTGGTACGTCGTTCAGGCGTTTT...,RNAInter,NaN,NaN,NaN,NaN,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,cgacaaaaggtaaagctaccgttgcttttgcccgtgaagcgcgtac...,TTTGCGAACTTATCGGTTTCGTATTGAGGGCAATAAACTCTCGAGG...,aacccagcgatcaaaaaagcggcgatttaatcgttgcacaaggcgt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,ryhB,msrA,"chromosome:NC_000913.3, Gene ID:2847761","chromosome:NC_000913.3, Gene ID:948734",sRNA,mRNA,No Interaction,['NA'],['NA'],GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase,3580927..3581016,4441538..4442176,reverse,reverse,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,atggtgtcgctctcccgaaaatcggtcattggggttaaggttgtaa...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,aggtactgtaaagccgccagagggttaaaattcagacagctgaaaa...
374,gcvB,hdeA,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948025",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase,2942696..2942901,3656408..3656740,forward,reverse,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,atcgtaatatcctcaactataaagtgaaagagccgtcacgaatcaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,gccaacgcagattgtgcgttcaccagtgacaaagccgctacagcgc...
375,gcvB,hdeB,"chromosome:NC_000913.3, Gene ID:2847720","chromosome:NC_000913.3, Gene ID:948026",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase,2942696..2942901,3655966..3656292,forward,reverse,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,attttgtaacccattcaatatagagattatatatgaacaggaggaa...,GGTAGAGCGCCATTGCATTCTTTAGCGCCTCGATCCCTTTATTTTT...,cataaacagggcgcttcccatgccaattaatacatgtgtgcgtaac...
376,Esr41,fliC,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",trans-encoded antisense RNA,mRNA,Induction,['NA'],['NA'],GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBa

In [5]:
data['Interactor 1'].unique()

array(['dsrA', 'oxyS', 'sgrS', 'ryhB', 'yzgL', 'ygaY', 'yagJ', 'sdsR',
       'micF', 'micC', 'istR', 'omrA', 'omrB', 'rprA', 'rseX', 'rybB',
       'Spot_42', 'cyaR', 'dicF', 'gadY', 'symR', 'sokB', 'sokC', 'gcvB',
       'rdlD', 'smpB', 'fnrS', 'sibC', 'arcZ', 'chiX', 'mgrR', 'sibB',
       'mcaS', 'ohsC', 'OrzO', 'sibD', 'sibE', 'sibA', 'IpeX', 'rdlA',
       'rdlB', 'rdlC', 'arrS', 'mntS', 'AfaR', 'Esr41'], dtype=object)

In [6]:
data[data['Interactor 1'] == 'chiX']

,Interactor 1,Interactor 2,ID 1,ID 2,Category 1,Category 2,Regulation,Binding position 1,Binding position 2,Sequence 1,Sequence 2,Source DB,Genome position 1,Genome position 2,Strand 1,Strand 2,Sequence before 1,Sequence before 2,Sequence after 1,Sequence after 2
234,chiX,rpoS,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:947210",sRNA,mRNA,No Interaction,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,sRNATarBase,507204..507287,2866559..2867551,forward,reverse,gtttcagccggatcgtgctcgtaggtatggatttgaaacgaaatct...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...,ttagcgcaacatttgcggcaaattcccttctccatacaggtgtagt...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...
290,chiX,citA,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945233",trans-encoded antisense RNA,mRNA,Repression,['46..57'],['-37..-26'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGTTGCAGCTTAACGAGAATAAACAGTTTGCATTTTTCCAAAGAC...,sRNATarBase,507204..507287,652235..653893,forward,forward,gtttcagccggatcgtgctcgtaggtatggatttgaaacgaaatct...,GCCTGTGACCGACGTACTGGGGGATACGCAGATAAACTCAT,ttagcgcaacatttgcggcaaattcccttctccatacaggtgtagt...,GCCTGTGACCGACGTACTGGGGGATACGCAGATAAACTCAT
308,chiX,chiP,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945296",trans-encoded antisense RNA,mRNA,Repression,['81..92'],['45..56'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGCGTACGTTTAGTGGCAAACGTAGTACGCTGGCGCTGGCTATCG...,sRNATarBase,507204..507287,708334..709740,forward,forward,gtttcagccggatcgtgctcgtaggtatggatttgaaacgaaatct...,GCCTGTGACGGGGTCGGTTTGTACACCGATAAACTCAT,ttagcgcaacatttgcggcaaattcccttctccatacaggtgtagt...,GCCTGTGACGGGGTCGGTTTGTACACCGATAAACTCAT
341,chiX,chbC,"chromosome:NC_000913.3, Gene ID:5061500","chromosome:NC_000913.3, Gene ID:945982",trans-encoded antisense RNA,mRNA,Repression,['NA'],['NA'],ACACCGTCGCTTAAAGTGACGGCATAATAATAAAAAAATGAAATTC...,ATGAGTAATGTTATTGCATCGCTTGAAAAGGTACTCCTCCCTTTTG...,sRNATarBase,507204..507287,1819856..1821214,forward,reverse,gtttcagccggatcgtgctcgtaggtatggatttgaaacgaaatct...,atagaaaaatacccttaaaaaccgcaatttaaatattgcggtattg...,ttagcgcaacatttgcggcaaattcccttctccatacaggtgtagt...,ttgctgcggcaaaatcgccctgcttcgcctgtttcagtgccgcata...


In [7]:
len(data)

378

## Cut out the RNA's that are longer than 300bp

In [8]:
data = data[data['Sequence 1'].apply(len) <= 300]
len(data)

375

In [9]:
len(data['Interactor 1'].unique())

43

# Config

In [10]:
config = {
    "experiment": {
        "purpose": "tests",
        "no_visualisations": False,
        "no_numerical": False,
        "debug_mode": False
    },
    "system_type": "RNA",
    "include_prod_deg": False,
    "interaction_simulator": {
        "name": "IntaRNA",
        "postprocess": True,
        "multithread": 0,
        "simulator_kwargs": {
            "outcsvcols": "id1,id2,E,E_norm,subseqDB,Pu1,Pu2"
        }
    },
    "mutations_args": {
        "algorithm": "all",
        "mutation_counts": 0,
        "mutation_nums_within_sequence": [1],
        "mutation_nums_per_position": 1,
        "concurrent_species_to_mutate": "single_species_at_a_time",
        "seed": 0
    },
    "signal": {
        "inputs": [data['Interactor 1'].iloc[0]],
        "outputs": [],
        "function_name": "step_function",
        "function_kwargs": {
            "impulse_center": 1,
            "impulse_halfwidth": 5,
            "target": 2
        }
    },
    "simulation": {
        "dt0": 0.1,
        "t0": 0,
        "t1": 100,
        "tmax": 2000,
        "solver": "diffrax",
        "use_batch_mutations": True,
        "interaction_factor": 1,
        "batch_size": 200,
        "max_circuits": 2000,
        "device": "cpu",
        "threshold_steady_states": 0.05,
        "use_rate_scaling": True
    },
    "molecular_params_factor": 1,
    "molecular_params": {
        "avg_mRNA_per_cell": 100,
        "cell_doubling_time": 1200,
        "creation_rate": 2.35,
        "starting_copynumbers": 200,
        "degradation_rate": 0.01175,
        "association_binding_rate": 1000000
    }
}



# Simulate

In [11]:
# from synbio_morpher.utils.results.writer import DataWriter
# data_writer = DataWriter(purpose=config['experiment']['purpose'], out_location= 'data/04_distribution_mutational_robustness/2024_01_29_140340')
data_writer = None
config, data_writer = script_preamble(config, data_writer)
config_file = expand_config(config=config)
config_file = prepare_config(config_file)

In [12]:
# Load previously simulated IntaRNA script
fn = './data/05_orthogonality_mRNA_official/2024_01_04_105924/inter_data_raw.json'
sim_data_l = load_json_as_dict(fn)
default_vals = {
    'id1': '', 'id2': '', 'E': 0.0, 'E_norm': 0.0, 'subseqDB': '', 'bpList': '', 'hybridDPfull': '', 'Pu1': 0.0, 'Pu2': 0.0, 'seedPu1': '', 'seedPu2': '', 'seedStart1': '', 'seedStart2': '', 'seedEnd1': '', 'seedEnd2': ''
}
for s, v in sim_data_l.items():
    for s2 in v:
        if v[s2] == {}:
            v[s2].update(default_vals)

pattern = r'\s*\([^)]*\)'
for i in [1, 2]:
    data[f'Binding position {i}'] = list(map(lambda raw: flatten_listlike(
        [[[int(re.sub(pattern, '', bp)) for bp in bps.split('..')]
          for bps in re.sub(pattern, '', par).split('; ')] for par in raw if par.split('..')[0].strip('-').isnumeric()]), data[f'Binding position {i}'].apply(str).str.split("'")))


def get_srnas(data):
    srnas, mrnas = [], []
    filt_srna1 = data['Category 1'].isin(
        ['sRNA', 'trans-encoded antisense RNA', 'protein binding small RNA'])
    filt_srna2 = data['Category 2'].isin(
        ['sRNA', 'trans-encoded antisense RNA', 'protein binding small RNA']) & (~data['Category 1'].isin(['Protein']))
    for i in [1, 2]:
        filt = filt_srna1 if i == 1 else filt_srna2
        for s in data[f'Interactor {i}'].unique():
            if s in data[data[f'Category {i}'] == 'mRNA'][f'Interactor {i}'].unique():
                mrnas.append(s)
            elif s in data[filt][f'Interactor {i}'].unique():
                srnas.append(s)
    return srnas, mrnas


srnas, mrnas = get_srnas(data)
sim_data_l = {s: {m: vv for m, vv in v.items() if m in mrnas and (
    len(data[(data['Interactor 1'] == s) & (data['Interactor 2'] == m)]) or
    len(data[(data['Interactor 2'] == s) & (data['Interactor 1'] == m)])
    )} for s, v in sim_data_l.items() if s in srnas}

/tmp/ipykernel_897355/232467086.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'Binding position {i}'] = list(map(lambda raw: flatten_listlike(


## Mutate

Only mutate the parts of the sequence that are already binding, otherwise the whole simulation will take way too long.

First preprocess the binding sites curated from the interaction databases. If there are no documented binding 

In [13]:
k1 = list(sim_data_l.keys())[0]
k2 = list(sim_data_l[k1].keys())[0]
sim_data_l[k1][k2]

{'id1': 'dsrB',
 'id2': 'dsrA',
 'E': '-1.11',
 'E_norm': '-0.108103',
 'subseqDB': '131CCCUGAGG&71CCUCAGGG',
 'Pu1': '0.00164667',
 'Pu2': '3.86378e-08'}

In [14]:
len(sim_data_l)

42

In [15]:
sim_data_l.keys() == (set(sim_data_l.keys()) - set(sim_data_l[k1].keys()))

True

In [16]:
sim_data_l[k1].keys() == (set(sim_data_l[k1].keys()) - set(sim_data_l.keys()))

True

### Make binding site range

In [17]:
def int_from_string_nums(text):
    return int(''.join(re.findall(r'\d', text)))

def str_nonum(text):
    return re.sub(r'\d', '', text)

bad_k1s = []
bad_k2s = []
for k1, v in sim_data_l.items():
    if k1 not in data[['Interactor 1', 'Interactor 2']].melt()['value'].to_list():
        bad_k1s.append(k1)
    for k2, vv in v.items():
        if k2 not in data[['Interactor 1', 'Interactor 2']].melt()['value'].to_list():
            bad_k2s.append(k2)
        for i, vvv in enumerate(vv['subseqDB'].split('&')):
            dna_start = int_from_string_nums(vvv)
            sim_data_l[k1][k2][f'binding_range_id{i + 1}'] = (dna_start, dna_start + len(str_nonum(vvv)) - 1)

In [18]:
for k1 in bad_k1s:
    del sim_data_l[k1]

for k1 in sim_data_l:
    for k2 in bad_k2s:
        if k2 in sim_data_l[k1]:
            del sim_data_l[k1][k2]

In [19]:

def get_labels(vsim, data):
    assert vsim['id1'] in data['Interactor 1'].to_list() + data['Interactor 2'].to_list(), 'id1 is not in data'
    assert vsim['id2'] in data['Interactor 1'].to_list() + data['Interactor 2'].to_list(), 'id2 is not in data'
    i = 1 if (vsim['id1'] in data['Interactor 1'].to_list()) else 2
    j = 2 if (vsim['id2'] in data['Interactor 2'].to_list()) else 1
    if i == j:
        # print(f'Interactions {i} == {j} for species {vsim["id1"]} and {vsim["id2"]}')
        pass
    ri = data[(data[f'Interactor {i}'] == vsim['id1'])]
    rj = data[(data[f'Interactor {j}'] == vsim['id2'])]
    r = data[(data[f'Interactor {i}'] == vsim['id1']) & (data[f'Interactor {j}'] == vsim['id2'])]
    id_srna = 1 if (ri[f'Category {i}'] != 'mRNA').any() else 2
    id_mrna = 2 if (rj[f'Category {j}'] == 'mRNA').any() else 1
    # assert r[f'Category {j}'] != 'mRNA', 'Label the sRNA better'
    if id_srna == id_mrna:
        # print(f'The sRNA and mRNA ids are the same: {i} == {j} for species {vsim["id1"]} and {vsim["id2"]}')
        pass
    return i, j, r, ri, rj, id_srna, id_mrna


def make_seqs(i, j, r, ri, rj):
    rs = [ri, rj]
    seqs = [''] * 2
    for idx, p in enumerate([i, j]):
        seqs[idx] = rs[idx][f'Sequence {p}'].iloc[0]
        if r[f'Binding position {p}'].any():
            mins = r[f'Binding position {p}'].apply(lambda x: np.min(x) if x else 1)
            if mins.min() < 0:
                seqs[idx] = r[f'Sequence before {p}'].iloc[np.argmin(mins)] + seqs[idx]
        seqs[idx] = seqs[idx].upper().replace('T', 'U')
    return seqs


def make_mutations(circuits, seqs, vsim, id_srna, id_mrna):
    mutations = {k: [] for k in ['mutation_name', 'mutation_types', 'positions']}
    for i, mutation_pos in enumerate(np.arange(vsim[f'binding_range_id{id_srna}'][0], vsim[f'binding_range_id{id_srna}'][1])):
        i_mut = 0
        for nuc in ['A', 'C', 'G', 'U']:
            try:
                if seqs[id_srna - 1][mutation_pos] == nuc:
                    continue
            except:
                print('bad')
            mutation_name = 'mutation_' + str(i) + '-' + str(i_mut)
            circuits[vsim[f'id{id_srna}']][vsim[f'id{id_mrna}']][mutation_name] = {
                vsim[f'id{id_srna}']: seqs[id_srna - 1][:mutation_pos] + nuc + seqs[id_srna - 1][mutation_pos+1:],
                vsim[f'id{id_mrna}']: seqs[id_mrna - 1]
            }
            i_mut += 1
            mutations['mutation_name'].append(mutation_name)
            mutations['mutation_types'].append([get_mutation_type_mapping('RNA')[seqs[id_srna - 1][mutation_pos]][nuc]])
            mutations['positions'].append([mutation_pos])
    mutations = pd.DataFrame.from_dict(mutations)
    mutations['count'] = 1
    mutations['template_file'] = ''
    return circuits, mutations

circuits_all = {}
mutations_all = {}
for k1, v in sim_data_l.items():
    for k2, vsim in v.items():
        i, j, r, ri, rj, id_srna, id_mrna = get_labels(vsim, data)
        if id_srna == id_mrna:
            continue
        circuits_all.setdefault(vsim[f'id{id_srna}'], {})
        
        seqs = make_seqs(i, j, r, ri, rj)
        mutation_name = 'ref_circuit'
        circuits_all[vsim[f'id{id_srna}']].setdefault(vsim[f'id{id_mrna}'], {})
        circuits_all[vsim[f'id{id_srna}']][vsim[f'id{id_mrna}']][mutation_name] = {
            vsim[f'id{id_srna}']: seqs[id_srna - 1],
            vsim[f'id{id_mrna}']: seqs[id_mrna - 1]
        }
        circuits_all, mutations = make_mutations(circuits_all, seqs, vsim, id_srna, id_mrna)
        mutations_all.setdefault(vsim[f'id{id_srna}'], {})
        mutations_all[vsim[f'id{id_srna}']].setdefault(vsim[f'id{id_mrna}'], {})
        mutations_all[vsim[f'id{id_srna}']][vsim[f'id{id_mrna}']] = mutations
        


In [20]:
circuits_all['AfaR']['afaD'].keys()

dict_keys(['ref_circuit', 'mutation_0-0', 'mutation_0-1', 'mutation_0-2', 'mutation_1-0', 'mutation_1-1', 'mutation_1-2', 'mutation_2-0', 'mutation_2-1', 'mutation_2-2', 'mutation_3-0', 'mutation_3-1', 'mutation_3-2', 'mutation_4-0', 'mutation_4-1', 'mutation_4-2', 'mutation_5-0', 'mutation_5-1', 'mutation_5-2', 'mutation_6-0', 'mutation_6-1', 'mutation_6-2', 'mutation_7-0', 'mutation_7-1', 'mutation_7-2', 'mutation_8-0', 'mutation_8-1', 'mutation_8-2', 'mutation_9-0', 'mutation_9-1', 'mutation_9-2', 'mutation_10-0', 'mutation_10-1', 'mutation_10-2', 'mutation_11-0', 'mutation_11-1', 'mutation_11-2', 'mutation_12-0', 'mutation_12-1', 'mutation_12-2', 'mutation_13-0', 'mutation_13-1', 'mutation_13-2', 'mutation_14-0', 'mutation_14-1', 'mutation_14-2', 'mutation_15-0', 'mutation_15-1', 'mutation_15-2', 'mutation_16-0', 'mutation_16-1', 'mutation_16-2', 'mutation_17-0', 'mutation_17-1', 'mutation_17-2', 'mutation_18-0', 'mutation_18-1', 'mutation_18-2', 'mutation_19-0', 'mutation_19-1', '

In [21]:
circuits_all['chiX'].keys()

dict_keys(['rpoS', 'chiP', 'citA', 'chbC'])

## Make circuits

In [22]:
tot_subcircuits = sum([sum([len(l) for m, l in ms.items()]) for s, ms in circuits_all.items()])
print('Total circuits: ', tot_subcircuits)


Total circuits:  24103


In [23]:
for srna, d_mrna in circuits_all.items():
    data_writer.subdivide_writing(srna)
    for mrna, muts in d_mrna.items():
        mutations = mutations_all[srna][mrna]
        data_writer.output(data=mutations, out_name='mutations',
                           out_type='csv', subfolder=mrna)

In [24]:
# circuits = []
# for srna, d_mrna in circuits_all.items():
#     for mrna, muts in d_mrna.items():
#         circuit_name = srna + '_' + mrna
#         mutations = mutations_all[srna][mrna]
#         mutations['circuit_name'] = circuit_name
#         data_writer.output(data=mutations, out_name='mutations',
#                            out_type='csv', subfolder=circuit_name)

#         for mutation_name, d_mutation in muts.items():

#             config_file['data'] = d_mutation
#             config_file['name'] = mutation_name
#             config_file = expand_model_config(
#                 config_file, config_file, sample_names=[srna, mrna])

#             kwargs = compose_kwargs(prev_configs=None, config=config_file)
#             kwargs['name'] = circuit_name
#             circuit = instantiate_system(kwargs)
#             circuit.subname = mutation_name
#             circuits.append(circuit)

# len(circuit.model.species)

In [25]:
# # for circuit in circuits:
# # num_subcircuits = [len(flatten_nested_dict(
# #     c.mutations)) + 1 for c in circuits_all]
# # tot_subcircuits = sum(num_subcircuits)
# tot_subcircuits = len(circuits)

# print('Total circuits: ', tot_subcircuits)
# # print(num_subcircuits)

In [26]:
# A simulation where RNAs have lengths [1071, 828] (aroF, YagJ) takes 20s
# A simulation where RNAs have lengths [1122, 828] (aroF, YagJ) takes 28s
# A simulation where RNAs have lengths [1758, 828] (fliC, YagJ) takes 29s

n_segs = 24
ave_sim_per_inter = 30 #s

print('Number of different interactions:', len(mutations_all))
print('Number of subcircuits:', tot_subcircuits)
print('Estimated time: ', tot_subcircuits * ave_sim_per_inter / 60 / 60 / 24, 'days')
print('Estimated time: ', tot_subcircuits * ave_sim_per_inter / 60 / 60, 'hours')
print('Estimated time with multi: ', tot_subcircuits * ave_sim_per_inter / 60 / 60 / 24 / n_segs, 'days')
print('Estimated time with multi: ', tot_subcircuits * ave_sim_per_inter / 60 / 60 / n_segs, 'hours')

Number of different interactions: 38
Number of subcircuits: 24103
Estimated time:  8.369097222222221 days
Estimated time:  200.85833333333332 hours
Estimated time with multi:  0.3487123842592592 days
Estimated time with multi:  8.369097222222221 hours


In [70]:
i = 0

start_i = int(np.floor((tot_subcircuits / n_segs) * np.arange(0, i+1))[i])
end_i = int(np.floor((tot_subcircuits / n_segs) * np.arange(1, i+2))[i])

start_i, end_i

# end_i = int(np.floor(tot_subcircuits / n_segs) * (i+1))
# circuits = circuits[i*int(len(circuits)/n_segs):(i+1)*int(len(circuits)/n_segs)]
# circuits = circuits[:5]

(3012, 4017)

In [71]:
# # {c.name: (len(c.get_input_species()[0].physical_data), len(c.get_input_species()[1].physical_data)) for c in circuits}
# data[data['Interactor 2'] == 'fliC']['Sequence 2'].iloc[0]
# {c.name: list(len(s.physical_data) for s in c.get_input_species()) for c in circuits}

In [72]:
# from synbio_morpher.utils.common.setup import compose_kwargs
# modeller = CircuitModeller(result_writer=data_writer, config=config_file)
# curr_subcircuits = []
# for i, circuit in enumerate(circuits[:3]):
#     curr_subcircuits.append(modeller.load_mutations(circuit))
# curr_subcircuits = flatten_listlike(curr_subcircuits)


## Run sim

In [73]:
data_writer.write_dir

'data/04_distribution_mutational_robustness/2024_01_29_140340'

In [79]:
def simulate_IntaRNA_local(query: dict,
                           target: str,
                           sim_kwargs={}):
    fn1 = data_writer.output(data=query, out_type='fasta',
                             out_name='query', byseq=True, return_path=True)
    fn_targets = data_writer.output(
        data=target, out_type='fasta', out_name='target', byseq=True, return_path=True)
    sim_kwargs['query'] = fn1
    sim_kwargs['target'] = fn_targets

    def run(query: str, target: str, qidxpos0: int, tidxpos0: int, outcsvcols: str, threads: int, n: int = 1,
            param_file: str = '', extra_params: list = [], raw_stdout: bool = False):
        p = Popen(['IntaRNA', '-q', query, '-t', target,
                   '--outMode=C', f'--outcsvcols={outcsvcols}',
                   f'--qIdxPos0={qidxpos0}',
                   f'--tIdxPos0={tidxpos0}',
                   f'--outNumber={n}',
                   f'--threads={threads}', param_file]
                  + extra_params, stdout=PIPE, stderr=PIPE, universal_newlines=True)
        stdout, stderr = p.communicate()
        return process_raw_stdout(stdout)

    res = run(**sim_kwargs)
    os.remove(fn1)
    os.remove(fn_targets)
    return res


circ_counter = 0
for s, ms in circuits_all.items():
    
    # sim_data.setdefault(s, {})
    data_writer.subdivide_writing(s)
    for m, muts in ms.items():
        
        # sim_data.setdefault(m, {})
        data_writer.subdivide_writing(m, safe_dir_change=False)
        t = datetime.now()
        sim_data = {}
        for mut_name, mut in muts.items():
            if circ_counter < start_i:
                circ_counter += 1
                continue
            elif circ_counter == end_i:
                break 
            
            sim_data.setdefault(mut_name, {})

            sim_data[mut_name] = simulate_IntaRNA_local(query={s: mut[s]},
                                                        target={m: mut[m]},
                                                        sim_kwargs=config['interaction_simulator']['simulator_kwargs'])
            circ_counter += 1
        
        if sim_data:
            print('Finished ', s, m, ' in ', (datetime.now() - t).total_seconds(), ' s')

            if os.path.isfile(os.path.join(data_writer.write_dir, 'inter_data_raw.json')):
                sim_data.update(load_json_as_dict(os.path.join(data_writer.write_dir, 'inter_data_raw.json')))
            data_writer.output(data=sim_data, out_type='json',
                            out_name='inter_data_raw', overwrite=False)
        data_writer.unsubdivide_last_dir()
    data_writer.unsubdivide()

Finished  sgrS manX  in  5.771625  s


In [76]:
# modeller = CircuitModeller(result_writer=data_writer, config=config_file)
# circuits_sim = modeller.batch_circuits(
#     circuits=circuits[4:8],
#     write_to_subsystem=True, batch_size=config_file['simulation'].get('batch_size', 100),
#     methods={
#         "compute_interactions": {},
#         # "compute_interactions_batch": {'batch': True},
#         # "write_interactions": {},
#         # "init_circuits": {'batch': True},
#         # "simulate_signal_batch": {'ref_circuit': None,
#         #                           'batch': True},
#         "write_results": {'no_visualisations': config_file['experiment'].get('no_visualisations', True),
#                           'no_numerical': config_file['experiment'].get('no_numerical', False)}
#     })

## Save

In [107]:
# fn = 'data/tests/2024_01_20_041436/dsrA_fhlA/mutations.csv'
# m = pd.read_csv(fn)
# m

In [108]:
# source_dirs = [data_writer.top_write_dir]
# # source_dirs = ['data/tests/2024_01_18_175210/']
# tables = load_multiple_as_list(inputs_list=source_dirs, load_func=b_tabulate_mutation_info, 
#             data_writer=data_writer)
# saves = pd.concat(tables, axis=0)
# saves

# Compare

In [109]:
# print(f'Number simulated: {len(saves)}')